In [8]:
%%capture
!pip install bitsandbytes
!pip install accelerate
!pip install --upgrade transformers

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import transformers
import torch
from transformers import AutoModelForCausalLM

In [3]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

In [4]:
messages = [
    {"role": "system", "content": "You are an expert chef"},
    {"role": "user", "content": "Write the recipe for pastitsio, a Greek dish"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

In [5]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [10]:
outputs = pipeline(
    prompt,
    max_new_tokens=1000,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Pastitsio, the beloved Greek pasta dish! It's a classic, hearty, and satisfying casserole that combines the best of Greek cuisine with the comfort of a warm, satisfying pasta bake. Here's my recipe for this iconic dish:

**Pastitsio Recipe**

**Ingredients:**

For the pasta:

* 1 pound (450g) penne or macaroni
* Salt, for cooking

For the meat sauce:

* 1 pound (450g) ground beef
* 1 medium onion, finely chopped
* 2 cloves of garlic, minced
* 1 cup (250ml) red wine
* 1 cup (250ml) beef broth
* 2 tablespoons tomato paste
* 2 tablespoons olive oil
* 1 teaspoon dried oregano
* 1 teaspoon ground cinnamon
* Salt and pepper, to taste

For the béchamel sauce:

* 2 tablespoons butter
* 2 tablespoons all-purpose flour
* 1 cup (250ml) milk
* 1/2 cup (120ml) grated Parmesan cheese
* Salt and pepper, to taste

For the topping:

* 1 cup (120g) grated Parmesan cheese
* 1/4 cup (30g) breadcrumbs
* 2 tablespoons olive oil

**Instructions:**

1. **Cook the pasta:** Bring a large pot of salted water to 